# Toward an ARABIC Stop-Words List Generation
One of the most important preprocessing techniques is the removal of functional words which affects the performance of text mining tasks. In this [paper](https://www.researchgate.net/publication/306364790_Toward_an_ARABIC_Stop-Words_List_Generation), a statistical approach is presented to extract Arabic stop-words list. The extracted list was compared to a general list. The comparison yield an improvement in an ANN based classifier using the generated stop-words list over the general list.


stop words are words which are filtered out before or after processing of natural language data.

In [2]:
import pandas as pd
import glob

import unicodedata as ud

import numpy as np
import pickle
import matplotlib.pyplot as plt
from collections import Counter

from sklearn import feature_extraction, model_selection

### Explore Data

In [141]:
#get all csv files and concatenate them into single dataframe
path = r'../data/'
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

data = pd.concat(li, axis=0, ignore_index=True)
data.head(n=-1)

,article
0,بمجرد أن تكون لك عملية مالية أو غيرها داخل أغل...
1,قبل فترة ليست بالقصيرة صدرت قرارات بخصوص فتح ا...
2,هناك مثل شعبي عقلاني يقول (راحت السكرة وجت الف...
3,انتهت قبل عدة أيام مناسك الحج التي أداها الحجا...
4,إن لانتشار الأشجار في المدن مزايا عديدة، منها ...
...,...
23792,تعيد شبكات التواصل فكرة الساحة العامة الرئيسية...
23793,كتب داليبور روهاك، باحث زميل في معهد اميركان ا...
23794,مضى الآن على مقاطعة الدول الأربع لقطر ما يزيد ...
23795,ما بين البَصْرة ونجد، في عشرينيات القرن الماضي...


In [142]:
#remove NaN
data = data[data['article'].notna()]

In [143]:
#remove punctuation
for i in range(data.shape[0]):
    data.iloc[i][0] = ''.join(c for c in data.iloc[i][0] if not ud.category(c).startswith('P'))
data

,article
0,بمجرد أن تكون لك عملية مالية أو غيرها داخل أغل...
1,قبل فترة ليست بالقصيرة صدرت قرارات بخصوص فتح ا...
2,هناك مثل شعبي عقلاني يقول راحت السكرة وجت الفك...
3,انتهت قبل عدة أيام مناسك الحج التي أداها الحجا...
4,إن لانتشار الأشجار في المدن مزايا عديدة منها ز...
...,...
23793,كتب داليبور روهاك باحث زميل في معهد اميركان ان...
23794,مضى الآن على مقاطعة الدول الأربع لقطر ما يزيد ...
23795,ما بين البَصْرة ونجد في عشرينيات القرن الماضي ...
23796,في محاضرته واجب الشباب المسلم اليوم التي ألقاه...


In [144]:
#split sentences to words
words = pd.DataFrame()
for i in range(data.shape[0]):
    words = words.append(data.iloc[i][0].split())
words

,0
0,بمجرد
1,أن
2,تكون
3,لك
4,عملية
...,...
339,الفساد
340,جعجعة
341,ولا
342,يرى


### Step 1: Word Frequency Calculation

In [146]:
wordsFrequency = pd.DataFrame(Counter(words[0]).most_common(50), columns=['word', 'frequency'])
wordsFrequency

,word,frequency
0,في,356927
1,من,286652
2,على,172219
3,أن,146083
4,إلى,99920
5,التي,80677
6,عن,67941
7,لا,65888
8,ما,65712
9,أو,54998


### Step 2: Mean and variance Calculation

In [248]:
FreqOfWordInDocs = []
numberOfDistinctWordsInDocs = []
propOfWordInDocs = []

In [249]:
path = r'../data/'
all_files = glob.glob(path + "/*.csv")

li = []
file_number = 0

for filename in all_files:
    data = pd.read_csv(filename, index_col=None, header=0)
    
    #remove NaN
    data = data[data['article'].notna()]

    #remove punctuation
    for i in range(data.shape[0]):
        data.iloc[i][0] = ''.join(c for c in data.iloc[i][0] if not ud.category(c).startswith('P'))

    #split sentences to words
    words = pd.DataFrame()
    for i in range(data.shape[0]):
        words = words.append(data.iloc[i][0].split())

    #add number of distinct words in each doc to a list
    numberOfDistinctWordsInDocs.append(words[0].unique().shape[0])

    #word frequency in each Doc
    FreqOfWord = pd.DataFrame(Counter(words[0]).most_common(100), columns=['word', 'frequency'])
    wordsFrequencyDict = dict()
    for i in range(FreqOfWord.shape[0]):
             wordsFrequencyDict[FreqOfWord.word[i]] = FreqOfWord.frequency[i]

    #add number of frequency of most common words in each doc to a list
    FreqOfWordInDocs.append(wordsFrequencyDict) 

    #probability of each 100 common word in each doc
    wordsProbabilityDict = dict()
    for i in wordsFrequencyDict :
             wordsProbabilityDict[i] = wordsFrequencyDict[i]/numberOfDistinctWordsInDocs[file_number]

    #add probability of most common words in each doc to a list
    propOfWordInDocs.append(wordsProbabilityDict)
    
    file_number+=1

In [250]:
FreqOfWordInDocs

[{'في': 35032,
  'من': 29276,
  'على': 16404,
  'أن': 14138,
  'التي': 9000,
  'إلى': 8552,
  'أو': 7259,
  'لا': 6940,
  'ما': 6604,
  'عن': 6220,
  'مع': 4755,
  'الذي': 4708,
  'هذا': 4697,
  'هذه': 4329,
  'كل': 4221,
  'ذلك': 3281,
  'المملكة': 3108,
  'هو': 3059,
  'كان': 2732,
  'بين': 2703,
  'لم': 2701,
  'الله': 2483,
  'السعودية': 2413,
  'ولا': 2258,
  'كما': 2126,
  'العالم': 2112,
  'تلك': 2105,
  'حتى': 2087,
  'هي': 1913,
  'كانت': 1884,
  'بعد': 1869,
  'خلال': 1861,
  'بعض': 1859,
  'أي': 1746,
  'غير': 1734,
  'بل': 1668,
  'إلا': 1656,
  'وهو': 1627,
  'فيها': 1547,
  'قبل': 1544,
  'قد': 1536,
  'العمل': 1527,
  'لها': 1492,
  'أكثر': 1487,
  'بها': 1462,
  'إن': 1459,
  'العربية': 1453,
  'عليه': 1450,
  'له': 1445,
  'الدول': 1426,
  'ومن': 1422,
  'حيث': 1360,
  'وفي': 1312,
  'يكون': 1303,
  'بن': 1299,
  'اليوم': 1289,
  'به': 1280,
  'يمكن': 1275,
  'المجتمع': 1265,
  'العام': 1264,
  'وهي': 1257,
  'هناك': 1219,
  'فيه': 1188,
  'السعودي': 1170,
  'مثل': 115

In [251]:
numberOfDistinctWordsInDocs

[139729, 144065, 135548, 109931, 194022, 87428, 194432, 188122]

In [252]:
propOfWordInDocs

[{'في': 0.25071388187133664,
  'من': 0.20951985629325337,
  'على': 0.11739867887124363,
  'أن': 0.1011815729018314,
  'التي': 0.06441039440631507,
  'إلى': 0.06120418810697851,
  'أو': 0.05195056144393791,
  'لا': 0.04966757079775852,
  'ما': 0.04726291607325609,
  'عن': 0.04451473924525331,
  'مع': 0.034030158378003134,
  'الذي': 0.03369379298499238,
  'هذا': 0.03361506916960688,
  'هذه': 0.030981399709437554,
  'كل': 0.03020847497656177,
  'ذلك': 0.02348116711634664,
  'المملكة': 0.022243056201647474,
  'هو': 0.021892377387657536,
  'كان': 0.019552133057561424,
  'بين': 0.019344588453363298,
  'لم': 0.019330275032384114,
  'الله': 0.017770112145653372,
  'السعودية': 0.017269142411382034,
  'ولا': 0.016159852285495494,
  'كما': 0.01521516650086954,
  'العالم': 0.015114972554015272,
  'تلك': 0.015064875580588138,
  'حتى': 0.014936054791775509,
  'هي': 0.01369078716658675,
  'كانت': 0.013483242562388623,
  'بعد': 0.013375891905044766,
  'خلال': 0.01331863822112804,
  'بعض': 0.0133043248

#### Calculate Mean of Probability (MP)

In [253]:
MP = dict()

In [257]:
for i in range(wordsFrequency.shape[0]):
    key = wordsFrequency.word[i]
    MP[key] = 0
    for j in range(file_number):
        if key in propOfWordInDocs[j]:
            MP[key] += propOfWordInDocs[j][key]
    MP[key] /= file_number

In [258]:
MP

{'في': 0.2914919081470928,
 'من': 0.23474608589267099,
 'على': 0.14012712744779,
 'أن': 0.11976114186261148,
 'إلى': 0.07964854063417309,
 'التي': 0.06555220966082144,
 'عن': 0.055855313814814914,
 'لا': 0.05331684867175042,
 'ما': 0.052951728974971676,
 'أو': 0.04449214418543106,
 'مع': 0.04438480187423934,
 'الذي': 0.04254578811913884,
 'هذا': 0.03702227354525748,
 'هذه': 0.031032883914035213,
 'كان': 0.02995438986022642,
 'لم': 0.026860457312116844,
 'كل': 0.027739217841469294,
 'ذلك': 0.025503614441094835,
 'هو': 0.024960412798031177,
 'بين': 0.024915697099442014,
 'بعد': 0.022366354916815143,
 'فى': 0.01925896317096227,
 'كانت': 0.01759536122990834,
 'ولا': 0.017528746369737603,
 'العالم': 0.017719353310724224,
 'كما': 0.016723251516920043,
 'حتى': 0.016081497517489145,
 'غير': 0.014867509409431871,
 'قبل': 0.01490206473001765,
 'الله': 0.01232866178059628,
 'أي': 0.014116199144456876,
 'هي': 0.013622251260758271,
 'قد': 0.012832374710935839,
 'وهو': 0.01390117138933919,
 'إيران':

#### Calculate Variance of Probability (VP)

In [265]:
totalWordsInAllDocs = 0
for i in range(wordsFrequency.shape[0]):
    totalWordsInAllDocs += wordsFrequency.frequency[i]
totalWordsInAllDocs

2298738

In [268]:
VP = dict()
for i in range(wordsFrequency.shape[0]):
    key = wordsFrequency.word[i]
    VP[key] = 0
    for j in range(file_number):
        if key in propOfWordInDocs[j]:
            VP[key] += (propOfWordInDocs[j][key] - MP[key])
    VP[key] = VP[key] ** 2    
    VP[key] /= file_number

In [269]:
VP

{'في': 7.800016274768305e-33,
 'من': 3.851859888774472e-34,
 'على': 0.0,
 'أن': 0.0,
 'إلى': 7.2824226022142356e-34,
 'التي': 6.018531076210112e-36,
 'عن': 2.407412430484045e-35,
 'لا': 9.62964972193618e-35,
 'ما': 9.62964972193618e-35,
 'أو': 2.1666711874356403e-34,
 'مع': 9.62964972193618e-35,
 'الذي': 6.018531076210112e-36,
 'هذا': 6.018531076210112e-36,
 'هذه': 0.0,
 'كان': 7.372700568357387e-35,
 'لم': 9.62964972193618e-35,
 'كل': 3.046881357331369e-35,
 'ذلك': 1.2187525429325477e-34,
 'هو': 1.8431751420893468e-35,
 'بين': 1.3541694921472752e-35,
 'بعد': 5.416677968589101e-35,
 'فى': 0.0007418153248409621,
 'كانت': 9.4039548065783e-36,
 'ولا': 1.504632769052528e-36,
 'العالم': 3.385423730368188e-36,
 'كما': 6.018531076210112e-36,
 'حتى': 1.8431751420893468e-35,
 'غير': 0.0,
 'قبل': 5.416677968589101e-35,
 'الله': 7.59979506501677e-05,
 'أي': 2.407412430484045e-35,
 'هي': 1.504632769052528e-36,
 'قد': 2.0583730090233234e-05,
 'وهو': 3.76158192263132e-35,
 'إيران': 6.16502214123789e